### Imports

In [1]:
import pandas as pd
from my_connector import MySQLConnector
import numpy as np
import matplotlib as plt

## Leemos el dataset de paradas de metro

In [2]:
df_stops = pd.read_csv('./Datasets/stopsmetro.csv')
df_stops.head()

 nos quitamos las columnas no deseadas

In [3]:
df_stops.drop(columns=['stop_timezone', 'wheelchair_boarding', 'stop_url', ' location_type'], inplace=True)

Como está dando un error extraño, inspeccionamos las columnas para saber cual es el problema

In [4]:
# Inspeccionar los nombres de las columnas para identificar espacios adicionales o caracteres inusuales
column_names = [col.strip() for col in df_stops.columns]

# Mostrar los nombres de las columnas después de eliminar los espacios adicionales
column_names

Sustituimos los nombres para que coincidan y nos quitamos las columnas no deseadas

In [5]:
# Eliminar específicamente las columnas 'stop_timezone' y 'wheelchair_boarding' después de ajustar los nombres de las columnas
df_stops.columns = column_names
df_stops = df_stops.drop(['stop_timezone', 'wheelchair_boarding', 'stop_url', 'location_type'], axis=1)

# Mostrar las primeras filas del DataFrame corregido para confirmar la eliminación de las columnas
df_stops.head()

Vamos a realizar la conexion a la BBDD y ejecutar el query para montar la estructura relacional de la misma con tablas dcon la siguiente información:
1. **Tabla Stations**
Columnas:
station_id (VARCHAR, PRIMARY KEY): Basado en stop_id.
station_code (INT): Basado en stop_code.
name (VARCHAR): Basado en stop_name.
description (VARCHAR): Basado en stop_desc.
latitude (FLOAT): Basado en stop_lat.
longitude (FLOAT): Basado en stop_lon.
zone_id (VARCHAR): Basado en zone_id.

2. **Tabla LocationTypes**
Columnas:
location_type_id (INT, PRIMARY KEY)
description (VARCHAR)

3. **Tabla StationLocationTypes**
Columnas:
station_id (VARCHAR): Clave foránea de Stations.
location_type_id (INT): Clave foránea de LocationTypes.
4. **Tabla ParentStations**
Columnas:
parent_station_id (VARCHAR, PRIMARY KEY)
station_id (VARCHAR): Clave foránea de Stations.

In [34]:
# Crear una instancia del conector
connector = MySQLConnector(host="localhost", user="root", password="root", database="tnt")
# Conectar a la base de datos
connector.connect()

# Lista de consultas SQL para crear las tablas
queries = [
    """
    CREATE TABLE `Parent_Stations` (
    `station_id` VARCHAR(255) PRIMARY KEY,
    `name` VARCHAR(255),
    `description` VARCHAR(255),
    `latitude` DOUBLE,
    `longitude` DOUBLE,
    `zone_id` VARCHAR(255)
);

"""
    ,
    """
    CREATE TABLE `Stops` (
    `stop_id` VARCHAR(255) PRIMARY KEY,
    `stop_code` INT,
    `name` VARCHAR(255),
    `description` VARCHAR(255),
    `latitude` DOUBLE,
    `longitude` DOUBLE,
    `zone_id` VARCHAR(255),
    `parent_station_id` VARCHAR(255),
    FOREIGN KEY (`parent_station_id`) REFERENCES `Parent_Stations`(`station_id`)
);
    """
]

# Ejecutar cada consulta para crear las tablas
for query in queries:
    connector.execute_query(query)

Extraemos valores únicos de parent_station (excluyendo valores nulos o vacíos)

In [35]:
unique_parent_stations = df_stops[df_stops['parent_station'].notna()]['parent_station'].unique()
unique_parent_stations = pd.DataFrame(unique_parent_stations)
unique_parent_stations.rename(columns={0:'station_id'}, inplace=True)
unique_parent_stations

Extraer información completa para las estaciones padre

In [36]:
# Extraer información completa para las estaciones parentales
parent_stations_info = df_stops[df_stops['stop_id'].isin(unique_parent_stations['station_id'])]
parent_stations_info

Vemos que solo recibimos 4 entradas de estaciones padre que existen en el csv, pero hay más referenciadas. Procederemos a agregarlas a la tabla de estaciones padre para no perder la integridad de los datos y que estén relacionadas con sus estaciones padre

In [37]:
for station_id in unique_parent_stations['station_id']:
    if station_id not in parent_stations_info['stop_id'].values:
        # Crea un registro nuevo con valores nulos o predeterminados
        new_row = {'stop_id': station_id, 'stop_name': 'Nombre desconocido', 'stop_desc': 'Descripción no disponible',
                   'stop_lat': 0.0, 'stop_lon': 0.0, 'zone_id': 'Desconocido'}
        # Añade este nuevo registro a parent_stations_info
        parent_stations_info = parent_stations_info.append(new_row, ignore_index=True)
parent_stations_info

In [38]:
parent_stations_info.drop(columns=['parent_station','stop_code'], inplace=True)
parent_stations_info.head()

Introducimos los datos del dataframe parent_stations_info en la tabla Parent_Stations

In [39]:
# Iterar sobre cada fila del DataFrame para preparar y ejecutar las consultas SQL
for index, row in parent_stations_info.iterrows():
    query = f"""
    INSERT INTO `Parent_Stations` (station_id, name, description, latitude, longitude, zone_id)
    VALUES ('{row['stop_id']}', '{row['stop_name']}', '{row['stop_desc']}', {row['stop_lat']}, {row['stop_lon']}, '{row['zone_id']}');
    """
    connector.execute_query(query)

Preparamos un df para introducir a la tabla Stops sin redundar con la tabla Parent_stations

In [40]:
stops_to_insert = df_stops[~df_stops['stop_id'].isin(parent_stations_info['stop_id'])]

stops_to_insert['parent_station'] = stops_to_insert['parent_station'].fillna('NULL')
stops_to_insert.head()

Introducimos los datos del dataframe stops_to_insert en la tabla Stops tratando con los nulos en parent_station ya que es un FK

In [41]:
# Iterar sobre cada fila del DataFrame para preparar y ejecutar las consultas SQL
for index, row in stops_to_insert.iterrows():
    # Verificar si parent_station debe ser tratado como NULL en SQL
    parent_station_value = 'NULL' if row['parent_station'] == 'NULL' else f"'{row['parent_station']}'"

    # Preparar la consulta SQL teniendo en cuenta el tratamiento correcto de NULL
    query = f"""
    INSERT INTO `Stops` (stop_id, stop_code, name, description, latitude, longitude, zone_id, parent_station_id)
    VALUES ('{row['stop_id']}', {row['stop_code']}, '{row['stop_name']}', '{row['stop_desc']}', {row['stop_lat']}, {row['stop_lon']}, '{row['zone_id']}', {parent_station_value});
    """

    connector.execute_query(query)

connector.close()


## Distritos
Creamos una tabla distritos para poder ubicar generalmente cada estacion de metro

In [42]:
# Crear una instancia del conector
connector = MySQLConnector(host="localhost", user="root", password="root", database="tnt")
# Conectar a la base de datos
connector.connect()
queries =['''
DROP TABLE IF EXISTS Distritos;
''',
'''
CREATE TABLE Distritos (
  codigo_distrito int NOT NULL,
  nombre_distrito varchar(50) DEFAULT NULL,
  PRIMARY KEY (codigo_distrito)
);
''',

'''
INSERT INTO Distritos VALUES (0,'DESCONOCIDO'),(1,'CENTRO'),(2,'ARGANZUELA'),(3,'RETIRO'),(4,'SALAMANCA'),(5,'CHAMARTÍN'),(6,'TETUÁN'),(7,'CHAMBERÍ'),(8,'FUENCARRAL-EL PARDO'),(9,'MONCLOA-ARAVACA'),(10,'LATINA'),(11,'CARABANCHEL'),(12,'USERA'),(13,'PUENTE DE VALLECAS'),(14,'MORATALAZ'),(15,'CIUDAD LINEAL'),(16,'HORTALEZA'),(17,'VILLAVERDE'),(18,'VILLA DE VALLECAS'),(19,'VICÁLVARO'),(20,'SAN BLAS-CANILLEJAS'),(21,'BARAJAS');
'''
]

for query in queries:
    connector.execute_query(query)

### Agregamos columnas distritos a las tablas Parent_Stations y Stops

In [44]:
queries=["""
ALTER TABLE parent_stations
ADD COLUMN codigo_distrito int,
ADD CONSTRAINT fk_distrito_parents FOREIGN KEY (codigo_distrito) REFERENCES distritos(codigo_distrito)
""",
         """
ALTER TABLE stops
ADD COLUMN codigo_distrito int,
ADD CONSTRAINT fk_distrito_stops FOREIGN KEY (codigo_distrito) REFERENCES distritos(codigo_distrito)
"""]

for query in queries:
    connector.execute_query(query)

Leemos ahora datos específicos en las tablas ejecutando sentencias como SELECT, UPDATE o VIEW

## Casos de Uso de las Tablas

### Consultar todas las paradas junto con su estación parental
Seleccionamos los identificadores y nombres de las paradas junto con sus coordenadas geográficas. Además, vamos a recoger la información correspondiente de sus estaciones principales asociadas si existen. Para las paradas que no tienen una estación principal asignada, aún se mostrarán en los resultados con los detalles de la estación principal como nulos

In [15]:
query ="""
SELECT s.stop_id, s.name AS stop_name, s.latitude, s.longitude, p.station_id AS parent_station_id, p.name AS parent_station_name
FROM Stops s
LEFT JOIN `Parent_Stations` p ON s.parent_station_id = p.station_id;
"""
df_padres_hijos = connector.fetch_data_as_df(query)
df_padres_hijos

### Encontrar paradas sin estación parental
Obtenemos los identificadores, nombres y descripciones de las paradas que no están asignadas a ninguna estación principal.

In [16]:
query= '''
SELECT stop_id, name, description
FROM Stops
WHERE parent_station_id IS NULL;
'''
df_stops_without_parent= connector.fetch_data_as_df(query)
df_stops_without_parent

### Listar distritos con el número total de paradas
De la tabla distritos, seleccionamos las que comparten el mismo codigo de distrito en la tabla stops y las agrupammos por distrito

In [45]:
query= '''
SELECT d.nombre_distrito, COUNT(s.stop_id) AS num_stops
FROM distritos d
LEFT JOIN stops s ON d.codigo_distrito = s.codigo_distrito
GROUP BY d.nombre_distrito;
'''
df_dist_count= connector.fetch_data_as_df(query)
df_dist_count

### Consulta de todas las paradas en un distrito específico
Vamos a buscar y listar todas las paradas que están en el distrito X, en este caso: 'FUENCARRAL-EL PARDO', mostrando su nombre, descripción y coordenadas geográficas. Para ello, uniremos la tabla de paradas con la tabla de distritos basándonos en su código de distrito común.

In [50]:
query= '''
SELECT s.name, s.description, s.latitude, s.longitude
FROM stops s
JOIN distritos d ON s.codigo_distrito = d.codigo_distrito
WHERE d.nombre_distrito = 'FUENCARRAL-EL PARDO';
'''
df_stops_in_dist = connector.fetch_data_as_df(query)
df_stops_in_dist

### Buscar paradas cercanas dentro de un radio específico (por ejemplo, 0.01 grados latitud/longitud):
Identificamos paradas de autobús cercanas entre sí en términos de su ubicación geográfica. Comparamos cada parada con todas las otras usando sus coordenadas de latitud y longitud. Establecemos un umbral de cercanía de 0.01 grados, que es aproximadamente 1 km o menos dependiendo de la ubicación geográfica, para considerar dos paradas como "cercanas". La consulta excluye comparaciones de una parada con sí misma para evitar falsos positivos.

In [51]:
query= '''
SELECT s1.name AS stop_name, s2.name AS nearby_stop_name
FROM stops s1, stops s2
WHERE ABS(s1.latitude - s2.latitude) < 0.01 AND ABS(s1.longitude - s2.longitude) < 0.01 AND s1.stop_id <> s2.stop_id;
'''
df_nearby_stops= connector.fetch_data_as_df(query)
df_nearby_stops